<a href="https://colab.research.google.com/github/pravallika12345678/FMML_Assignments/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

# ANSWERS

1.The percentage of the validation set, relative to the overall dataset, can have a notable impact on your model's training and the accuracy of the validation set. This split is a crucial part of the process, as it determines how much data is used for training and how much is used for assessing the model's performance. Here's what happens when you increase or reduce the percentage of the validation set:

**Increase Percentage of Validation Set**:

1. **Pros**:

   - **Better Validation**: A larger validation set means more data is available for evaluating the model's performance during training. This can lead to a more accurate estimate of how well the model generalizes to unseen data.
   
   - **Reduced Risk of Overfitting**: With a larger validation set, there's a greater chance of detecting overfitting early in the training process. This is because the model has less training data to potentially memorize and more validation data to identify when it's not generalizing well.

2. **Cons**:

   - **Smaller Training Set**: Increasing the validation set size inherently reduces the amount of data available for training. This can be problematic if you have a limited dataset to begin with, as it may hinder the model's ability to learn complex patterns in the data.

**Reduce Percentage of Validation Set**:

1. **Pros**:

   - **Larger Training Set**: A smaller validation set leaves more data available for training, which can be beneficial, especially if you have limited training data.

   - **Faster Training**: Smaller validation sets can speed up the training process since less time is spent on validation checks.

2. **Cons**:

   - **Less Reliable Validation**: With a smaller validation set, your estimate of model performance may be less reliable. It can be more sensitive to the specific random split of data and may not accurately reflect how well the model generalizes.

   - **Risk of Overfitting**: A smaller validation set might not effectively detect overfitting, allowing the model to fit noise in the training data without being penalized.

In practice, the choice of the percentage of the validation set is often a trade-off between having a sufficiently large validation set to obtain reliable performance estimates and having enough data for training. There is no one-size-fits-all answer, as the ideal split ratio depends on factors like the size of your dataset, the complexity of your model, and your specific goals.

Common split ratios include 70/30, 80/20, and 90/10, where the first number represents the percentage of the training set, and the second number represents the percentage of the validation set. You can experiment with different split ratios to find the one that strikes the right balance for your specific problem and dataset. Additionally, techniques like cross-validation can help provide more robust estimates of model performance when dealing with limited data for validation.

2.The size of the training and validation sets can have an impact on how well you can predict the accuracy of a model on the test set using the validation set. The relationship between these sets and the predictiveness of the validation set can be summarized as follows:

1. **Larger Training Set**:
   - If you allocate a larger portion of your data to the training set, it generally allows your model to learn more from the available data, potentially resulting in a better-performing model.
   - However, a smaller validation set may be less representative of the overall data distribution, and its accuracy may not reliably predict the model's performance on the test set. This is because the validation set has less data to assess the model's generalization.

2. **Larger Validation Set**:
   - A larger validation set provides a more reliable estimate of the model's performance during training. With more data for validation, you have a better chance of accurately assessing how well the model generalizes to unseen data.
   - However, this comes at the cost of a smaller training set, which may limit the model's ability to learn complex patterns from the data.

3. **Balanced Split**:
   - Striking a balance between the size of the training and validation sets is often a practical choice. Common split ratios include 70/30, 80/20, or 90/10, where the first number represents the percentage of the training set.
   - A balanced split aims to provide a sufficient amount of data for training while maintaining a reasonably sized validation set for accurate performance estimation.

The choice of how to allocate data between the training and validation sets depends on several factors, including the size of your dataset, the complexity of your model, and the goals of your project. It's essential to consider the trade-offs involved:

- If you have a large dataset, you can allocate a larger portion to both the training and validation sets, ensuring that both are sufficiently representative.

- If your dataset is small, you might need to prioritize one over the other. For example, you might choose a smaller validation set to leave more data for training when dealing with limited data.

- Cross-validation can be a helpful technique to address the limitations of small validation sets by repeatedly splitting the data into training and validation subsets to get a more robust estimate of model performance.

Ultimately, the predictiveness of the validation set concerning test set accuracy depends on the representativeness of the validation set, the amount of data allocated to it, and how well it captures the distribution of the test data. While a larger validation set generally provides more reliable estimates, it cannot fully compensate for issues like data bias or overfitting. The test set remains crucial for the final assessment of a model's generalization performance.



3.Determining the ideal percentage to reserve for the validation set to balance the two factors (having a sufficiently large training set and a reliable estimate of model performance) is context-dependent and can vary based on factors such as the size of your dataset, the complexity of your model, and your specific goals. However, there are some common practices and guidelines that you can consider:

1. **70/30 Split**: A common starting point is a 70/30 split, where you allocate 70% of your data to the training set and 30% to the validation set. This provides a reasonably balanced compromise between training data size and validation set reliability.

2. **80/20 Split**: Another frequently used split is 80/20, with 80% of the data for training and 20% for validation. This split gives a bit more data for training while still maintaining a reasonably sized validation set.

3. **90/10 Split**: If you have a large dataset, you might use a 90/10 split, which leaves 90% for training and 10% for validation. This can work well when you have an abundance of data.

4. **Stratified Split**: If your dataset has class imbalances, consider stratified splitting. This ensures that the class distribution in both the training and validation sets closely matches the overall dataset. Stratified splitting can be crucial for maintaining representative subsets.

5. **Cross-Validation**: When dealing with limited data or when you want a more robust estimate of model performance, consider using k-fold cross-validation. Common values for k include 5 or 10. Cross-validation repeatedly splits the data into training and validation subsets, helping to reduce the impact of randomness in a single split.

6. **Leave-One-Out Cross-Validation (LOOCV)**: For very small datasets, LOOCV, where each data point serves as the validation set once, can be considered, although it can be computationally expensive.

In practice, the choice of the split percentage often depends on practical considerations, including the number of samples in your dataset and computational resources. Smaller datasets may benefit from larger validation sets to provide more reliable performance estimates. Conversely, larger datasets can allocate a smaller percentage to validation without compromising the model's ability to learn.

It's important to remember that while the split percentage is essential, other factors like data quality, preprocessing, model choice, and hyperparameter tuning also influence model performance. Experimenting with different split ratios and validation techniques can help you find the right balance for your specific machine learning task.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


# ANSWERS

1.Yes, averaging the validation accuracy across multiple splits of your data can indeed provide more consistent and reliable results when evaluating the performance of a machine learning model. This process is commonly referred to as "cross-validation."

Cross-validation helps address the variability in the performance metrics that can occur due to the randomness in the initial data split into training and validation sets. By repeating the process of splitting the data into different subsets (folds) multiple times and then training and evaluating the model on these subsets, you get a better estimate of the model's generalization performance.

Here's how it works:

1. **K-Fold Cross-Validation**: One common technique is k-fold cross-validation, where the data is divided into k equally sized subsets or "folds." The model is trained and validated k times, with each fold serving as the validation set once while the others are used for training.

2. **Leave-One-Out Cross-Validation (LOOCV)**: In LOOCV, each data point is used as the validation set once, and the model is trained on the remaining data points. This process is repeated for all data points, resulting in n (the number of data points) iterations.

3. **Stratified Cross-Validation**: In situations where you have imbalanced classes, stratified cross-validation ensures that each fold maintains the same class distribution as the original data.

4. **Repeated Cross-Validation**: To further enhance reliability, you can repeat the cross-validation process multiple times with different random splits. This helps to account for variations in the data partitioning.

Averaging the validation accuracy or other performance metrics across these multiple runs of cross-validation provides a more stable estimate of how well your model is likely to perform on unseen data. It helps mitigate the impact of a single random split that might lead to overly optimistic or pessimistic results.

In summary, using cross-validation and averaging the results is a best practice in evaluating machine learning models, as it provides a more robust and consistent estimate of their performance and generalization capabilities.

2.Cross-validation, while valuable for assessing and comparing model performance on your training data, does not provide a direct estimate of test accuracy on unseen data. Instead, it offers an estimate of how well your model is likely to generalize to new, unseen data from the same distribution as your training data.

The primary purpose of cross-validation is to:

1. **Assess Model Performance**: Cross-validation helps you assess how well your model performs on different subsets of your training data. This can be helpful for model selection, hyperparameter tuning, and identifying potential issues like overfitting.

2. **Provide a More Robust Performance Estimate**: It helps in obtaining a more robust estimate of your model's expected performance on unseen data compared to a single train-test split. By averaging performance across multiple validation sets, it reduces the impact of data splitting randomness.

To estimate the test accuracy on truly unseen data (e.g., data that was not used during training or cross-validation), you should set aside a separate test dataset that your model has never seen. This test dataset serves as a proxy for how well your model is expected to perform in real-world applications.

The typical workflow is as follows:

1. **Split Your Data**: Initially, you split your dataset into three parts: a training set, a validation set (used for hyperparameter tuning and model evaluation during development), and a test set (held out for final evaluation).

2. **Train and Validate**: You use cross-validation on the training data to fine-tune your model and estimate its performance on various subsets of the training data.

3. **Select a Final Model**: Once you're satisfied with your model's performance on the validation data, you select a final model.

4. **Evaluate on Test Set**: You evaluate the final model's performance on the test set to get an estimate of how well it will perform on new, unseen data.

In summary, while cross-validation provides a more reliable estimate of model performance on your training data, the test set is crucial for estimating how well your model will generalize to completely new data, making it a better proxy for real-world performance.

3.The number of iterations or folds in cross-validation can have an effect on the reliability and stability of the estimate of your model's performance. In general, using more iterations typically results in a more robust and reliable estimate, up to a point. However, there are diminishing returns, and using too many iterations can be computationally expensive.

Here's how the number of iterations affects the estimate:

1. **Fewer Iterations (Folds)**:
   - Pros: Faster to compute.
   - Cons: The estimate may be more sensitive to the specific random splits of the data, which can lead to a higher degree of variability in the results. It might not capture the true generalization performance well.

2. **Moderate Number of Iterations (Folds)**:
   - Pros: Offers a reasonably robust estimate of model performance.
   - Cons: Strikes a balance between computation time and stability. This is often a good choice in practice.

3. **More Iterations (Folds)**:
   - Pros: Provides a more stable and reliable estimate of model performance by reducing the impact of data splitting randomness. It can give you a better understanding of how your model is likely to perform on average.
   - Cons: Requires more computational resources and time. There may be diminishing returns in terms of improved estimation as you increase the number of iterations.

The choice of the number of iterations (often referred to as "k" in k-fold cross-validation) should be made based on the size of your dataset, computational resources available, and the desired level of confidence in your estimate. Common choices for k include 5, 10, and sometimes 20. A value of 10 is often recommended as a good compromise between computational cost and estimation reliability.

It's essential to remember that while increasing the number of iterations generally provides a more reliable estimate, the quality of the estimate also depends on the representativeness of your data and how well it reflects the real-world distribution of the problem you're solving. Additionally, you should still reserve a separate test dataset for the final evaluation of your model's generalization performance.

4.Increasing the number of iterations in cross-validation can help mitigate the impact of having a very small training dataset or validation dataset to some extent, but it cannot completely compensate for the limitations of extremely small datasets. Here's how it works:

1. **Small Training Dataset**: If you have a very small training dataset, you are limited in the amount of data your model can learn from. Increasing the number of cross-validation iterations (k) can help ensure that your model gets exposed to different subsets of the limited training data, which can provide a more robust estimate of its performance. However, it does not magically increase the amount of training data you have. The fundamental issue of limited data for training still remains.

2. **Small Validation Dataset**: Having a small validation dataset can affect the reliability of your performance estimates because it may not provide a representative sample of the model's generalization performance. Increasing the number of cross-validation iterations can help by ensuring that you evaluate your model on different subsets of the small validation data. This can reduce the impact of the small validation dataset on your performance estimate.

While increasing the number of iterations is a good practice to stabilize your performance estimates, it does not solve the problem of insufficient data. If your training dataset is very small, you may encounter several challenges:

- **Model Complexity**: With a small training dataset, you are limited in the complexity of models you can effectively train. Complex models might overfit the limited data, leading to poor generalization.

- **Limited Feature Representation**: Small datasets may not adequately represent the diversity and complexity of the real-world data distribution, potentially leading to biased model performance estimates.

- **Hyperparameter Tuning**: It can be challenging to perform effective hyperparameter tuning with limited data, as there may not be enough information to make informed decisions about the best hyperparameter settings.

To address these issues, you might consider techniques such as data augmentation (if applicable), transfer learning, or using simpler models with fewer parameters. If possible, acquiring more data or collecting additional features can also significantly improve model performance and reliability. While cross-validation can help make the best use of the available data, it cannot fully compensate for data scarcity.